In [ ]:
from number_statistics import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

### Shows a histogram of the distribution of followers per user
Its purpose is to decide on the type of data structure would be best to use for saving the list of accounts in order

In [ ]:
count = count()
follows = count.keys() # x
frequency = count.values() # y

In [ ]:
# Plots a simple histogram showing the frequency of pictures that have a certain number of follows
plt.bar(follows, frequency, log=True)
plt.xlabel('Number of followers')
plt.ylabel('Frequency in dataset')


In [ ]:
# Plots a histogram which tells you the distribution of followers based on a certain power
# Note: the -1 in the graphs denote 0 followers
powers = [2,4,10]

for i, power in enumerate(powers, start=1):
    count_powers = magnitude_of_scale(count, base=power)
    follows = count_powers.keys() # x
    frequency = count_powers.values() # y
    
    plt.figure(i)
    plt.bar(follows, frequency, log=True)
    xlabel = 'Number of followers %d^n' % power
    plt.xlabel(xlabel)
    plt.ylabel('Frequency in dataset')
plt.show()